## 注意力机制Attention:

### 简介:

这一部分是Transformer模型的核心部分,以下部分逐步给出实现过程中可能用到的一些矩阵运算的原理， 以下代码均不需要大家实现,希望大家阅读代码以及下列文档中的信息:

https://arxiv.org/abs/1706.03762

https://jalammar.github.io/illustrated-transformer/

理解Attention的运行机制以及实现过程的数学技巧，完成最后的主文件中的HeadAttention(),MultiHeadAttention()部分。

我们虚构一组输入数据的Embedding用于这部分讲解：

In [1]:
import torch
from torch import nn
import torch.nn.functional as F
B, T, C = 1, 8, 16   ## B: batch size 一次训练的数据量, T: context length 前文token数, C: embedding length 隐变量长度
inputData = torch.rand(size=(B,T,C))

for i in range(T):
    print(f"Embedding of {i}th position:\n {inputData[0,i]}")


Embedding of 0th position:
 tensor([0.8780, 0.1966, 0.5001, 0.5708, 0.5584, 0.5402, 0.8622, 0.9725, 0.0555,
        0.0857, 0.7418, 0.5192, 0.9885, 0.4669, 0.8061, 0.8124])
Embedding of 1th position:
 tensor([0.9596, 0.9850, 0.8983, 0.0041, 0.4927, 0.6086, 0.2436, 0.4048, 0.1848,
        0.6848, 0.8204, 0.4509, 0.0788, 0.8161, 0.0703, 0.1991])
Embedding of 2th position:
 tensor([0.3875, 0.5733, 0.6702, 0.5349, 0.5495, 0.3688, 0.0275, 0.0074, 0.8870,
        0.5112, 0.4720, 0.0278, 0.1696, 0.5117, 0.3643, 0.0368])
Embedding of 3th position:
 tensor([0.1611, 0.8296, 0.2646, 0.9874, 0.6971, 0.2720, 0.6822, 0.6935, 0.2255,
        0.7172, 0.6284, 0.4187, 0.3870, 0.6030, 0.0955, 0.6509])
Embedding of 4th position:
 tensor([0.0292, 0.8401, 0.3817, 0.2062, 0.1839, 0.5465, 0.6254, 0.9140, 0.5523,
        0.0823, 0.6846, 0.0624, 0.7706, 0.3006, 0.8666, 0.3375])
Embedding of 5th position:
 tensor([0.5826, 0.1963, 0.7478, 0.0232, 0.7085, 0.1517, 0.2456, 0.7079, 0.3825,
        0.5051, 0.6555, 0.1

Attention从直观上可以理解为对前文各个位置信息的融合以获得当前语境所需的信息。 一个最简单的融合方式为对前文Embedding加权求和作为当前位置的信息。

我们计算第i个位置的融合后的embedding:

假设前i个位置的embedding的权重相同，均为1/i，即更新后第i个位置embedding为前文所有位置embedding的平均值：

In [2]:
def Attention_version1(contextEmbeddings):
    for i in range(T):
        context_embeddings = contextEmbeddings[0,:i+1,:] ## shape [i+1, C]
        new_embedding_for_i = torch.mean(context_embeddings,dim=0)
        contextEmbeddings[0,i] = new_embedding_for_i
    return contextEmbeddings

print("Embedding of Data after aggregate context embedding:\n", Attention_version1(inputData))

Embedding of Data after aggregate context embedding:
 tensor([[[0.8780, 0.1966, 0.5001, 0.5708, 0.5584, 0.5402, 0.8622, 0.9725,
          0.0555, 0.0857, 0.7418, 0.5192, 0.9885, 0.4669, 0.8061, 0.8124],
         [0.9188, 0.5908, 0.6992, 0.2874, 0.5255, 0.5744, 0.5529, 0.6886,
          0.1202, 0.3852, 0.7811, 0.4850, 0.5337, 0.6415, 0.4382, 0.5057],
         [0.7281, 0.4536, 0.6232, 0.4644, 0.5445, 0.4944, 0.4809, 0.5562,
          0.3542, 0.3274, 0.6650, 0.3440, 0.5639, 0.5400, 0.5362, 0.4516],
         [0.6715, 0.5177, 0.5218, 0.5775, 0.5814, 0.4703, 0.6446, 0.7277,
          0.1888, 0.3789, 0.7041, 0.4417, 0.6183, 0.5629, 0.4690, 0.6052],
         [0.6451, 0.5198, 0.5452, 0.4213, 0.4787, 0.5252, 0.6332, 0.7718,
          0.2542, 0.2519, 0.7153, 0.3705, 0.6950, 0.5024, 0.6232, 0.5425],
         [0.7374, 0.4125, 0.6062, 0.3908, 0.5662, 0.4593, 0.5699, 0.7374,
          0.2259, 0.3224, 0.7104, 0.3847, 0.6159, 0.4563, 0.5097, 0.5348],
         [0.6722, 0.4714, 0.5915, 0.4159, 0.5066, 0.

我们将上述的mean操作换为等价的矩阵运算，以i=3 为例：

new_embedding_for_3 = torch.mean(contextEmbeddings[0,:3+1],dim=0)

等价于(@ 是矩阵乘法):

new_embedding_for_3 = contextEmbeddings[0] @ torch.tensor([1/4,1/4,1/4,1/4,0,0,0,0])

In [3]:
def Attention_version2(contextEmbeddings):
    for i in range(T):
        weight = torch.cat((torch.ones(i+1) / (i+1),torch.zeros(T-i-1,dtype=torch.float)),dim=0)
        contextEmbeddings[0,i] = weight @ contextEmbeddings[0]
    return contextEmbeddings

print("Attention_version1 equivalent to Attention_version2: ",torch.all(Attention_version1(inputData) == Attention_version2(inputData)).item())

Attention_version1 equivalent to Attention_version2:  True


接下来我们用矩阵运算进一步简化上述运算，移除其中的for循环:

其中 weight = torch.tril(torch.ones(T,T)) 得到:

[[1., 0., 0., 0., 0., 0., 0., 0.],

 [1., 1., 0., 0., 0., 0., 0., 0.],
 
 [1., 1., 1., 0., 0., 0., 0., 0.],
 
 [1., 1., 1., 1., 0., 0., 0., 0.],
 
 [1., 1., 1., 1., 1., 0., 0., 0.],
 
 [1., 1., 1., 1., 1., 1., 0., 0.],
 
 [1., 1., 1., 1., 1., 1., 1., 0.],
 
 [1., 1., 1., 1., 1., 1., 1., 1.]]
 
表示前文的求和权重相同都为一。

weight = weight.masked_fill(weight==0,float("-inf"))

weight = F.softmax(weight)

这两行用于归一化weight,即每一次加权求和的权重和为1，具体详见Softmax公式,我们可得到：

[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],

[0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],

[0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],

[0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],

[0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],

[0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250]]


In [4]:
def Attention_version3(contextEmbeddings):
    B, T, C = contextEmbeddings.shape
    weight = torch.tril(torch.ones(T,T))
    print("weight of context embeddings:\n",weight)
    weight = weight.masked_fill(weight==0,float("-inf"))
    weight = F.softmax(weight,dim=1)
    print("weight of context embeddings after regularization:\n",weight)
    contextEmbeddings[0] = weight @ contextEmbeddings[0]
    return contextEmbeddings

print("Attention_version1 equivalent to Attention_version3: ",torch.all(Attention_version1(inputData) == Attention_version3(inputData)).item())

weight of context embeddings:
 tensor([[1., 0., 0., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1., 1., 1.]])
weight of context embeddings after regularization:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3333, 0.3333, 0.3333, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2500, 0.2500, 0.2500, 0.2500, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2000, 0.2000, 0.2000, 0.2000, 0.2000, 0.0000, 0.0000, 0.0000],
        [0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.1667, 0.0000, 0.0000],
        [0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.1429, 0.0000],
        [0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.1250, 0.125

最后，我们确定计算weight的方法，上述三个版本的weight都是假定所有前文信息的重要程度相同,在大语言模型中，我们希望有一个灵活的方式计算前文信息对应当前语境的重要程度，为此Transformer引入了Query，Key，Value:

其中Query可以理解为当前语境对于前文信息的需求，Key可以理解为前文包含信息的索引，Value为前文所包含的信息。

Query 和 Key 用来计算信息融合的weight.

如何计算Query和Key，并用他们计算weight对Value加权求和是这次实验的重点内容，这里不能给出大家具体代码，希望大家参见Attention is All you need原论文以及助教提供的文档最后的参考链接学习这部分。

利于Query和Key得出的是信息相关性，我们需要遮盖住下文的信息(生成第i个token时，只可以使用0到i-1处的信息)，并且要对相关性归一化使之可以作为weight。这里利于Attension_version3()中的结论给出如何对计算出来的相关性加掩码和归一化:


In [5]:
def weight_mask_and_normalization(weight):
    tril = torch.tril(torch.ones_like(weight))
    weight = weight.masked_fill(tril == 0, float("-inf"))
    weight = F.softmax(weight,dim=-1)
    return weight

weight = torch.rand(T,T)
print("weight before mask and normalize:\n",weight)
print("weight after mask and normalize:\n",weight_mask_and_normalization(weight))

weight before mask and normalize:
 tensor([[0.0624, 0.9833, 0.2526, 0.7022, 0.2681, 0.1070, 0.0305, 0.1322],
        [0.2717, 0.8660, 0.2748, 0.4028, 0.7648, 0.1013, 0.0679, 0.2302],
        [0.3137, 0.2909, 0.7530, 0.0161, 0.7766, 0.3774, 0.2168, 0.3018],
        [0.0939, 0.4920, 0.1087, 0.0793, 0.2914, 0.3732, 0.4475, 0.2975],
        [0.3837, 0.8869, 0.9575, 0.2391, 0.2009, 0.2381, 0.3572, 0.9512],
        [0.8092, 0.1143, 0.4656, 0.8370, 0.5649, 0.6914, 0.3454, 0.6255],
        [0.6277, 0.8041, 0.7433, 0.3916, 0.4300, 0.2967, 0.0705, 0.8678],
        [0.3658, 0.6495, 0.8745, 0.7044, 0.6595, 0.0515, 0.5808, 0.7380]])
weight after mask and normalize:
 tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3557, 0.6443, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2834, 0.2770, 0.4397, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2227, 0.3317, 0.2261, 0.2195, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1632, 0.2699, 0.2897, 0.1412, 0